##### Copyright 2024 Google LLC.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Explore vision capabilities with the Gemini API

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://ai.google.dev/gemini-api/docs/vision"><img src="https://ai.google.dev/static/site-assets/images/docs/notebook-site-button.png" height="32" width="32" />View on ai.google.dev</a>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/google/generative-ai-docs/blob/main/site/en/gemini-api/docs/vision.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/google/generative-ai-docs/blob/main/site/en/gemini-api/docs/vision.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
</table>

The Gemini API can run inference on images and videos passed to it. When passed an image, a series of images, or a video, Gemini can:

*   Describe or answer questions about the content
*   Summarize the content
*   Extrapolate from the content

This tutorial demonstrates some possible ways to prompt the Gemini API with
images and video input. All output is text-only.

## Setup

Before you use the File API, you need to install the Gemini API SDK package and configure an API key. This section describes how to complete these setup steps.

### Install the Python SDK and import packages

The Python SDK for the Gemini API is contained in the [google-generativeai](https://pypi.org/project/google-generativeai/) package. Install the dependency using pip.

In [1]:
!pip install -q -U google-generativeai

Import the necessary packages.

In [2]:
import google.generativeai as genai
from IPython.display import Markdown

### Set up your API key

The File API uses API keys for authentication and access. Uploaded files are associated with the project linked to the API key. Unlike other Gemini APIs that use API keys, your API key also grants access to data you've uploaded to the File API, so take extra care in keeping your API key secure. For more on keeping your keys
secure, see [Best practices for using API
keys](https://support.google.com/googleapi/answer/6310037).

Store your API key in a Colab Secret named `GOOGLE_API_KEY`. If you don't already have an API key, or are unfamiliar with Colab Secrets, refer to the [Authentication quickstart](https://github.com/google-gemini/gemini-api-cookbook/blob/main/quickstarts/Authentication.ipynb).

In [3]:
from google.colab import userdata
GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')

genai.configure(api_key=GOOGLE_API_KEY)

## Prompting with images

In this tutorial, you will upload images using the File API or as inline data and generate content based on those images.

### Technical details (images)
Gemini 1.5 Pro and Flash support a maximum of 3,600 image files.

Images must be in one of the following image data [MIME types](https://developers.google.com/drive/api/guides/ref-export-formats):

-   PNG - `image/png`
-   JPEG - `image/jpeg`
-   WEBP - `image/webp`
-   HEIC - `image/heic`
-   HEIF - `image/heif`

Each image is equivalent to 258 tokens.

While there are no specific limits to the number of pixels in an image besides the model’s context window, larger images are scaled down to a maximum resolution of 3072x3072 while preserving their original aspect ratio, while smaller images are scaled up to 768x768 pixels. There is no cost reduction for images at lower sizes, other than bandwidth, or performance improvement for images at higher resolution.

For best results:

*   Rotate images to the correct orientation before uploading.
*   Avoid blurry images.
*   If using a single image, place the text prompt after the image.

### Upload an image file using the File API

Use the File API to upload an image of any size. (Always use the File API when the combination of files and system instructions that you intend to send is larger than 20 MB.)

**NOTE**: The File API lets you store up to 20 GB of files per project, with a per-file maximum size of 2 GB. Files are stored for 48 hours. They can be accessed in that period with your API key, but cannot be downloaded from the API. It is available at no cost in all regions where the Gemini API is available.

Start by downloading this [sketch of a jetpack](https://storage.googleapis.com/generativeai-downloads/images/jetpack.jpg).

In [4]:
!curl -o jetpack.jpg https://storage.googleapis.com/generativeai-downloads/images/jetpack.jpg

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  349k  100  349k    0     0  1997k      0 --:--:-- --:--:-- --:--:-- 2006k


Upload the image using [`media.upload`](https://ai.google.dev/api/rest/v1beta/media/upload) and print the URI, which is used as a reference in Gemini API calls.

In [5]:
# Upload the file and print a confirmation.
sample_file = genai.upload_file(path="jetpack.jpg",
                            display_name="Jetpack drawing")

print(f"Uploaded file '{sample_file.display_name}' as: {sample_file.uri}")

Uploaded file 'Jetpack drawing' as: https://generativelanguage.googleapis.com/v1beta/files/ta72oiyqu4nt


The `response` shows that the File API stored the specified `display_name` for the uploaded file and a `uri` to reference the file in Gemini API calls. Use `response` to track how uploaded files are mapped to URIs.

Depending on your use case, you can also store the URIs in structures such as a `dict` or a database.

### Verify image file upload and get metadata

You can verify the API successfully stored the uploaded file and get its metadata by calling [`files.get`](https://ai.google.dev/api/rest/v1beta/files/get) through the SDK. Only the `name` (and by extension, the `uri`) are unique. Use `display_name` to identify files only if you manage uniqueness yourself.

In [6]:
file = genai.get_file(name=sample_file.name)
print(f"Retrieved file '{file.display_name}' as: {sample_file.uri}")

Retrieved file 'Jetpack drawing' as: https://generativelanguage.googleapis.com/v1beta/files/ta72oiyqu4nt


Depending on your use case, you can store the URIs in structures, such as a `dict` or a database.

### Prompt with the uploaded image and text

After uploading the file, you can make GenerateContent requests that reference the File API URI. Select the generative model and provide it with a text prompt and the uploaded image.

In [7]:
# Choose a Gemini API model.
model = genai.GenerativeModel(model_name="gemini-1.5-pro-latest")

# Prompt the model with text and the previously uploaded image.
response = model.generate_content([sample_file, "Describe how this product might be manufactured."])

Markdown(">" + response.text)

>While a fully functional jetpack backpack remains outside the realm of current technology, we can still explore how one might be manufactured if such technology were to exist:

**1. Materials:**

* **Body/Fabric:** Lightweight yet incredibly durable material like ballistic nylon or even carbon fiber weaves for the backpack exterior. 
* **Boosters:** 
    * **Housing:**  Heat-resistant ceramics or high-grade alloys to withstand extreme temperatures.
    * **Nozzles:**  Advanced refractory metals like tungsten or rhenium for their heat resistance.
* **Power Source:**
    * **Steam Generator:** High-pressure boiler system potentially made from titanium or similar strong, lightweight metals.
    * **Fuel:**  Efficient and clean-burning fuel storage and delivery system. This is the biggest hurdle – current fuels lack the energy density for sustained flight. 
* **Electronics:**
    * **Micro-turbines/Valves:**  Precision engineered for control and efficiency.
    * **Control System:**  Sophisticated flight computer, gyroscopes, and sensors for stability and maneuverability.
* **Comfort Features:**
    * **Padding:**  High-quality foam padding for the straps and back panel.
    * **Straps:**  Adjustable, reinforced straps.

**2. Manufacturing Process:**

* **High-Tech Manufacturing:**  Extensive use of CNC machining, 3D printing (potentially with metals and ceramics), and advanced welding techniques for the boosters and power source components.
* **Electronics Integration:**  Skilled technicians would carefully assemble and integrate the complex electronics, wiring, and control systems.
* **Assembly:**  A combination of automated processes and skilled labor would be required to assemble the jetpack components, attach them to the backpack body, and ensure all systems are integrated seamlessly.
* **Rigorous Testing:**  Each jetpack would undergo rigorous safety and performance testing in specialized facilities, including simulations of flight conditions.

**3. Challenges:**

* **Energy Density:**  Finding a fuel source with sufficient energy density to power sustained flight in a compact, lightweight package is the primary challenge. 
* **Heat Management:**  Steam-powered boosters would generate extreme heat. Efficient cooling and heat dissipation systems would be critical to prevent damage to the backpack and the wearer.
* **Safety:**  Ensuring the safe operation of a personal jetpack would be paramount. Redundant systems, fail-safes, and rigorous pilot training would be essential.

**4. Ethical and Regulatory Considerations:**

* **Legislation:**  New laws and regulations would be necessary to govern the use of personal jetpacks, addressing issues like air traffic control, licensing, and safety standards.
* **Environmental Impact:**  The environmental impact of the fuel source and emissions would require careful consideration.

While the jetpack backpack described is still a fictional concept, advancements in materials science, energy storage, and propulsion technology may one day make such a product possible. However, significant engineering hurdles remain before we can truly take to the skies with our backpacks. 


### Upload one or more locally stored image files

Alternatively, you can upload your own files. You can download and use our drawings of [piranha-infested waters](https://storage.googleapis.com/generativeai-downloads/images/piranha.jpg) and a [firefighter with a cat](https://storage.googleapis.com/generativeai-downloads/images/firefighter.jpg). First, save these files to your local directory.

Then click **Files** on the left sidebar. For each file, click the **Upload** button, then navigate to that file's location and upload it:

<img width=400 src="https://ai.google.dev/tutorials/images/colab_upload.png">

When the combination of files and system instructions that you intend to send is larger than 20 MB in size, use the File API to upload those files, as previously shown. Smaller files can instead be called locally from the Gemini API:


In [ ]:
# Choose a Gemini model.
model = genai.GenerativeModel(model_name="gemini-1.5-pro-latest")

prompt = "Extract the text from the image."

In [8]:
import PIL.Image
sample_file4 = PIL.Image.open('/content/2024-10-27 07_49_12-Elearning.jpg')

In [22]:
# !unzip -q "/content/2024-10-27 08_03_33-Elearning.zip" -d /content/images
!unzip -q "/content/2024-10-27 17_20_16-Elearning.zip" -d /content/images1


In [23]:
# prompt: above code is not print markdown in the output

import google.generativeai as genai
from IPython.display import Markdown
from google.colab import userdata
import PIL.Image
import os

# ... (rest of your existing code)

# ... (your existing code up to the point where you process images)

image_dir = "/content/images1/"  # Replace with the actual path to your images directory
prompt = "Describe this image" #Added a prompt variable
i = 0

if os.path.exists(image_dir):
  for filename in os.listdir(image_dir):
    print(filename)
    try:
      image = PIL.Image.open(os.path.join(image_dir, filename))
      response = model.generate_content([prompt, image])
      display(Markdown(">" + response.text)) # Use display for Markdown output
    except Exception as e:
      print(f"Error processing {filename}: {e}") # Handle potential errors

else:
  print(f"Directory '{image_dir}' not found.")

2024-10-27 18_52_25-Elearning.jpg


>The image shows a matching exercise with four responses to an opportunity (A-D) and four possible actions to take (1-4). The correct answers are already marked. 

Here are the matches in the correct order:

* **1. C:** Exploit the opportunity by eliminating uncertainty to ensure that it definitely occurs. 
* **2. A:** Enhance the opportunity by increasing the probability of it occurring and increasing its positive impact.
* **3. D:** Share the opportunity by allying with an external party to increase the likelihood of securing benefits, and agreeing to share the rewards.
* **4. B:** Simply accept the opportunity and do nothing, or have a contingency plan if it happens. 


1725528634546.jpg


>The image presents a list of AI-powered tools categorized as "PAID" and "FREE," aiming to aid users in various tasks. 

**PAID Tools:**

* **Ideation:** ChatGPT 4
* **Images:** Midjourney
* **Notes:** Fireflies
* **Video:** Runway
* **Transcript:** Podium AI
* **Speech-to-text:** Trint AI
* **Shorts:** Opusclip
* **Design:** Canva
* **Writer:** ChatGPT 4 
* **Avatar:** Synthesia

**FREE Tools:**

* **Ideation:** Claude AI
* **Images:** Bing Create
* **Notes:**  Laxis
* **Video:** Pika Labs
* **Transcript:** Auris AI
* **Speech-to-text:** Audiopen
* **Shorts:** 2Short AI
* **Design:** Microsoft Designer
* **Writer:** Writesonic
* **Avatar:** Fotor

The image is created by Shruti Mishra and encourages users to follow her for a high-resolution PDF and exclusive access to her newsletter. 


2024-10-27 18_16_16-Elearning.jpg


...

>The image is a screenshot of a multiple choice question about risk management techniques, specifically focusing on avoidance techniques. 

The question asks to match examples to the avoidance technique they best represent. The provided examples (left side) and techniques (right side) are:

**Examples:**
A. Revise an order to ensure all items can be delivered within 24 hours
B. Decide not to make a presentation
C. Ensure team members are paid competitive rates

**Techniques:**
A. Avoid the threat altogether
B. Adjust your plan to avoid the effects of the threat
C. Eliminate the cause of the threat

The screenshot shows the attempted answers. However, they are incorrect. The correct pairings would be:

* **A. Revise an order to ensure all items can be delivered within 24 hours  --  B. Adjust your plan to avoid the effects of the threat**
* **B. Decide not to make a presentation  -- A. Avoid the threat altogether**
* **C. Ensure team members are paid competitive rates  -- C. Eliminate the cause of the threat** 


2024-10-27 18_24_53-Elearning.jpg


>The image is a question about active and passive acceptance, presented as a matching exercise. It lists four different scenarios and asks the reader to classify them as examples of active or passive acceptance.

**The scenarios are:**

* **A:** Accept that some veteran employees will leave or retire, at which point you can begin looking for suitable replacements.
* **B:** Prepare a marketing budget to counter any new publicity drive by a competitor.
* **C:** Draw up a list of alternate suppliers in case any current supplier is unable to deliver.
* **D:** Accept that bad weather is likely to slow down construction of your new office building.

**The answer choices are:**

* **Active Acceptance:** B and C are marked as correct
* **Passive Acceptance:** D is marked as correct, with space for one more answer

**Analysis of the Answers:**

* **Active acceptance** involves taking preemptive action to mitigate the impact of a potential threat. Options B and C exemplify this by outlining steps taken in anticipation of possible events (competitor's marketing and supplier failure). 
* **Passive acceptance** means acknowledging a potential risk without actively preparing for it. Option D demonstrates this by acknowledging the potential delay due to weather without suggesting any mitigating actions.

The image is missing one correct answer for "Passive Acceptance." Option A, accepting employee turnover and planning to find replacements *only when it happens*, would be the other example of passive acceptance. 


2024-10-27 17_35_47-Elearning.jpg


>The image presents a matching exercise where the task is to correctly order responses to a business opportunity.

**Left Side:**

Four potential responses to a business opportunity are listed (A-D):

A. Acquire a struggling competitor to ensure market dominance.
B. Ensure a competitive bid price and plan for capacity expansion.
C. Partner with another company to enhance chances of winning a contract.
D. Agree to a client's proposal to provide helpdesk service.

**Right Side:**

The correctly ordered responses (1-4) are:

1. Acquire a struggling competitor (corresponds to option A on the left).
2. Ensure competitive bid price and capacity planning (corresponds to option B on the left).
3. Partner with another company (corresponds to option C on the left).
4. Agree to a client's proposal (corresponds to option D on the left).

**Solution:**

The lines and markings connect the options from the left side to their corresponding correct order on the right side, visually representing the solution to the matching exercise. 


In [10]:
# Choose a Gemini model.
model = genai.GenerativeModel(model_name="gemini-1.5-pro-latest")

prompt = "Extract the text from the image."

response = model.generate_content([prompt,  sample_file4])

Markdown(">" + response.text)

>**What is the role of GenAI in "augmentation" for project management?**

*Select the best answer below, then click submit.*

- Aids in complex, strategic tasks by providing deep insights and supporting decision-making
- Performing simple, routine tasks that require little to no strategic thought
- Replacing project managers in decision-making processes 


In [ ]:
import PIL.Image

sample_file_2 = PIL.Image.open('piranha.jpg')
sample_file_3 = PIL.Image.open('firefighter.jpg')

Note that these inline data calls don't include many of the features available via the File API, such as getting file metadata, [listing](https://colab.research.google.com/drive/19xeyIMZJIk7Zn9KW5_50iZYv8OfjApL5?resourcekey=0-3JZ6U8oAFX7hqeV7gAXshw#scrollTo=VosrkvAyrx-v&line=3&uniqifier=1), or [deleting](https://colab.research.google.com/drive/19xeyIMZJIk7Zn9KW5_50iZYv8OfjApL5?resourcekey=0-3JZ6U8oAFX7hqeV7gAXshw#scrollTo=diCy9BgjLqeS&line=1&uniqifier=1) files.

### Prompt with multiple images

You can provide the Gemini API with any combination of images and text that fit within the model's context window. This example provides one short text prompt and the three images previously uploaded.

In [ ]:
# Choose a Gemini model.
model = genai.GenerativeModel(model_name="gemini-1.5-pro-latest")

prompt = "Write an advertising jingle showing how the product in the first image could solve the problems shown in the second two images."

response = model.generate_content([prompt, sample_file, sample_file_2, sample_file_3])

Markdown(">" + response.text)

### Get bounding boxes

You can ask the model for the coordinates of bounding boxes for objects in images. For object detection, the Gemini model has been trained to provide
these coordinates as relative widths or heights in range `[0,1]`, scaled by 1000 and converted to an integer. Effectively, the coordinates given are for a
1000x1000 version of the original image, and need to be converted back to the dimensions of the original image.

In [ ]:
# Choose a Gemini model.
model = genai.GenerativeModel(model_name="gemini-1.5-pro-latest")

prompt = "Return a bounding box for the piranha. \n [ymin, xmin, ymax, xmax]"
response = model.generate_content([sample_file_2, prompt])

print(response.text)

To convert these coordinates to the dimensions of the original image:

1.    Divide each output coordinate by 1000.
1.    Multiply the x-coordinates by the original image width.
1.    Multiply the y-coordinates by the original image height.

## Prompting with video

In this tutorial, you will upload a video using the File API and generate content based on those images.

## Technical details (video)

Gemini 1.5 Pro and Flash support up to approximately an hour of video data.

Video must be in one of the following video format [MIME types](https://developers.google.com/drive/api/guides/ref-export-formats):
  -   `video/mp4`
  -   `video/mpeg`
  -   `video/mov`
  -   `video/avi`
  -   `video/x-flv`
  -   `video/mpg`
  -   `video/webm`
  -   `video/wmv`
  -   `video/3gpp`

The File API service currently extracts image frames from videos at 1 frame per second (FPS) and audio at 1Kbps, single channel, adding timestamps every second. These rates are subject to change in the future for improvements in inference.

**NOTE:** The finer details of fast action sequences may be lost at the 1 FPS frame sampling rate. Consider slowing down high-speed clips for improved inference quality.

Individual frames are 258 tokens, and audio is 32 tokens per second. With metadata, each second of video becomes ~300 tokens, which means a 1M context window can fit slightly less than an hour of video.

To ask questions about time-stamped locations, use the format `MM:SS`, where the first two digits represent minutes and the last two digits represent seconds.

For best results:

*   Use one video per prompt.
*   If using a single video, place the text prompt after the video.

### Upload a video file to the File API

**NOTE**: The File API lets you store up to 20 GB of files per project, with a per-file maximum size of 2 GB. Files are stored for 48 hours. They can be accessed in that period with your API key, but they cannot be downloaded using any API. It is available at no cost in all regions where the Gemini API is available.

The File API accepts video file formats directly. This example uses the short NASA film ["Jupiter's Great Red Spot Shrinks and Grows"](https://www.youtube.com/watch?v=JDi4IdtvDVE0). Credit: Goddard Space Flight Center (GSFC)/David Ladd (2018).

> "Jupiter's Great Red Spot Shrinks and Grows" is in the public domain and does not show identifiable people. ([NASA image and media usage guidelines.](https://www.nasa.gov/nasa-brand-center/images-and-media/))

Start by retrieving the short video:

In [ ]:
!wget https://storage.googleapis.com/generativeai-downloads/images/GreatRedSpot.mp4

Upload the video to the File API and print the URI.

In [ ]:
video_file_name = "GreatRedSpot.mp4"

print(f"Uploading file...")
video_file = genai.upload_file(path=video_file_name)
print(f"Completed upload: {video_file.uri}")

### Verify file upload and check state

Verify the API has successfully received the files by calling the [`files.get`](https://ai.google.dev/api/rest/v1beta/files/get) method.

**NOTE**: Video files have a `State` field in the File API. When a video is uploaded, it will be in the `PROCESSING` state until it is ready for inference. Only `ACTIVE` files can be used for model inference.

In [ ]:
import time

# Check whether the file is ready to be used.
while video_file.state.name == "PROCESSING":
    print('.', end='')
    time.sleep(10)
    video_file = genai.get_file(video_file.name)

if video_file.state.name == "FAILED":
  raise ValueError(video_file.state.name)

### Prompt with a video and text

Once the uploaded video is in the `ACTIVE` state, you can make `GenerateContent` requests that specify the File API URI for that video. Select the generative model and provide it with the uploaded video and a text prompt.

In [ ]:
# Create the prompt.
prompt = "Summarize this video. Then create a quiz with answer key based on the information in the video."

# Choose a Gemini model.
model = genai.GenerativeModel(model_name="gemini-1.5-pro-latest")

# Make the LLM request.
print("Making LLM inference request...")
response = model.generate_content([video_file, prompt],
                                  request_options={"timeout": 600})

# Print the response, rendering any Markdown
Markdown(response.text)

### Refer to timestamps in the content

You can use timestamps of the form `MM:SS` to refer to specific moments in the video.

In [ ]:
# Create the prompt.
prompt = "What are the examples given at 01:05 and 01:19 supposed to show us?"

# Choose a Gemini model.
model = genai.GenerativeModel(model_name="gemini-1.5-pro-latest")

# Make the LLM request.
print("Making LLM inference request...")
response = model.generate_content([prompt, video_file],
                                  request_options={"timeout": 600})
print(response.text)

### Transcribe video and provide visual descriptions

If the video is not fast-paced (given that frames are sampled at 1 per second), it's possible to transcribe the video with visual descriptions for each shot.

In [ ]:
# Create the prompt.
prompt = "Transcribe the audio, giving timestamps. Also provide visual descriptions."

# Choose a Gemini model.
model = genai.GenerativeModel(model_name="gemini-1.5-pro-latest")

# Make the LLM request.
print("Making LLM inference request...")
response = model.generate_content([prompt, video_file],
                                  request_options={"timeout": 600})
print(response.text)

## List files

You can list all uploaded files and their URIs using `files.list_files()`.

In [ ]:
# List all files
for file in genai.list_files():
    print(f"{file.display_name}, URI: {file.uri}")

## Delete files

Files are automatically deleted after 2 days. You can also manually delete them using `files.delete()`.

In [ ]:
genai.delete_file(video_file.name)
print(f'Deleted file {video_file.uri}')